In [1]:
import re
import time
import pickle
import os
import numpy as np
import pandas as pd
from itertools import *
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
from textwrap import dedent
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.feature_selection import VarianceThreshold
import datetime
import math
# from plotnine import *
from sklearn.metrics import brier_score_loss
from sklearn.utils import column_or_1d

In [3]:
df = pd.read_csv('../data/ne_cd_2_l2_2016_10_03_featureset_1335_scoring.csv', error_bad_lines=False)

b'Skipping line 265029: expected 512 fields, saw 614\nSkipping line 265073: expected 512 fields, saw 696\nSkipping line 265095: expected 512 fields, saw 769\n'
b'Skipping line 265249: expected 512 fields, saw 614\nSkipping line 265293: expected 512 fields, saw 696\nSkipping line 265315: expected 512 fields, saw 769\n'
/home/marzuqkhan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (224,253,318,461) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.columns.tolist()

['Unnamed: 0',
 'ispsa',
 'dwelling_type_single_family_dwelling_unit',
 'dwelling_type_multi_family_dwelling',
 'dwelling_type_null',
 'dwelling_unit_size_1_single_family_dwelling',
 'dwelling_unit_size_10_19',
 'dwelling_unit_size_5_9',
 'dwelling_unit_size_20_49',
 'dwelling_unit_size_50_100',
 'dwelling_unit_size_101',
 'dwelling_unit_size_2_duplex',
 'dwelling_unit_size_4',
 'dwelling_unit_size_3_triplex',
 'dwelling_unit_size_null',
 'estimated_income_amount',
 'hh_upscale_buyer',
 'hh_upscale_male_buyer',
 'hh_upscale_female_buyer',
 'hh_book_buyer',
 'hh_family_magazine',
 'hh_female_oriented_magazine',
 'hh_religious_magazine',
 'hh_gardening_magazine',
 'hh_culinary_interest_magazine',
 'hh_health_fitness_magazine',
 'hh_do_it_yourselfer_magazine',
 'hh_financial_magazine',
 'hh_religious_contributor',
 'hh_political_contributer',
 'hh_donates_environment_cause_u',
 'hh_donates_environment_cause_y',
 'hh_donates_environment_cause_null',
 'hh_donates_to_charity_y',
 'hh_donates

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330910 entries, 0 to 330909
Columns: 512 entries, Unnamed: 0 to lalvoterid
dtypes: float64(506), int64(1), object(5)
memory usage: 1.3+ GB


In [7]:
df.shape

(330910, 512)

In [8]:
df.loc[894]

Unnamed: 0                                             894
ispsa                                                    6
dwelling_type_single_family_dwelling_unit                1
dwelling_type_multi_family_dwelling                      0
dwelling_type_null                                       0
dwelling_unit_size_1_single_family_dwelling              1
dwelling_unit_size_10_19                                 0
dwelling_unit_size_5_9                                   0
dwelling_unit_size_20_49                                 0
dwelling_unit_size_50_100                                0
dwelling_unit_size_101                                   0
dwelling_unit_size_2_duplex                              0
dwelling_unit_size_4                                     0
dwelling_unit_size_3_triplex                             0
dwelling_unit_size_null                                  0
estimated_income_amount                             125000
hh_upscale_buyer                                        

In [9]:
df.isnull().sum().tolist()

[0,
 6096,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5531,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 144980,
 138122,
 120036,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 144980,
 208982,
 144980,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6275,
 6096,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6096,
 6096,
 6096,
 6096,
 6096,
 6096,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 